# 16. RNN과 어텐션을 사용한 자연어 처리

In [1]:
# 공통 모듈 임포트
import numpy as np
import os

# 깔금한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## 16.1 Char-RNN으로 셰익스피어 문체 생성하기

### 16.1.1 훈련 데이터셋 만들기

In [2]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

shakespeare_url = 'https://homl.info/shakespeare'
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character', standardize='lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

2023-10-30 11:11:36.049212: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-30 11:11:36.049239: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-30 11:11:36.049243: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-30 11:11:36.049317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-30 11:11:36.049552: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-10-30 11:11:36.199803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [5]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [6]:
def to_dataset(sequence, length, shuffle = False, seed = None, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size = 100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [10]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed = 42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

### 16.1.2 Char-RNN 모델 만들고 훈련하기

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model_ckpt = tf.keras.callbacks.ModelCheckpoint('my_shakespeare_model', monitor = 'val_accuracy', save_best_only=True)
history = model.fit(train_set, validation_data= valid_set, epochs=2, callbacks=[model_ckpt])

Epoch 1/2


2023-10-30 01:41:03.227740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 01:41:10.656082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 01:41:10.759386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  31247/Unknown - 755s 24ms/step - loss: 1.9545 - accuracy: 0.4181

2023-10-30 01:53:37.631505: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 1045853850338730159
2023-10-30 01:53:37.631687: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 17037774049988833223
2023-10-30 01:53:37.631693: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9007514600193704349
2023-10-30 01:53:37.631753: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8800234887924680155
2023-10-30 01:53:37.631761: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3072504827153520709
2023-10-30 01:53:37.631766: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6276434334016941993
2023-10-30 01:53:37.631919: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv it

INFO:tensorflow:Assets written to: my_shakespeare_model/assets


INFO:tensorflow:Assets written to: my_shakespeare_model/assets


31247/31247 [==============================] - 776s 25ms/step - loss: 1.9545 - accuracy: 0.4181 - val_loss: 1.9396 - val_accuracy: 0.4209
Epoch 2/2
31247/31247 [==============================] - 787s 25ms/step - loss: 1.8943 - accuracy: 0.4339 - val_loss: 1.9565 - val_accuracy: 0.4145


In [13]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [14]:
y_proba = shakespeare_model.predict(['To be or not to b'])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

2023-10-30 02:07:06.127252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 02:07:06.240412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 645ms/step


'e'

### 16.1.3 가짜 셰익스피어 텍스트 생성하기

In [15]:
def next_char(text, temperature = 1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logit = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logit, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [16]:
def extend_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)

    return text

In [17]:
tf.random.set_seed(42)

In [18]:
extend_text('To be or not to be', temperature=0.01)

1/1 [==============================] - 0s 149ms/step


2023-10-30 02:07:07.032339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 02:07:07.102589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 38ms/step


'To be or not to be the the the the the the the the the the the the t'

In [19]:
extend_text('To be or not to be', temperature=1)

1/1 [==============================] - 0s 30ms/step


'To be or not to bes\n\nibosie:\nberp, grecled,\na mance grrore me wilich'

In [20]:
extend_text('To be or not to be', temperature=100)

1/1 [==============================] - 0s 32ms/step


"To be or not to beg ,mt'&o3f:ady-$\nws-nse?pws&ertj-vberdjw!c-yjewznq"

### 16.1.4 상태가 있는 RNN

## 16.2 감성 분석

In [4]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = 'imdb_reviews',
    split = ['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True
)

tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

2023-10-30 14:02:38.055471: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-30 14:02:38.055489: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-30 14:02:38.055493: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-30 14:02:38.055551: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-30 14:02:38.055771: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode('utf-8'))
    print('레이블:', label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
레이블: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development w

2023-10-30 14:02:39.954376: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

2023-10-30 14:02:41.278466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [7]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2


2023-10-30 14:02:43.788497: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 14:02:44.036392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 14:02:44.205149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


704/704 [==============================] - ETA: 0s - loss: 0.6940 - accuracy: 0.4978

2023-10-30 14:04:30.184217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 14:04:30.317455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


704/704 [==============================] - 113s 159ms/step - loss: 0.6940 - accuracy: 0.4978 - val_loss: 0.6929 - val_accuracy: 0.5012
Epoch 2/2
704/704 [==============================] - 141s 200ms/step - loss: 0.6928 - accuracy: 0.5053 - val_loss: 0.6952 - val_accuracy: 0.5012


### 16.2.1 마스킹

### 16.2.2 사전 훈련된 임베딩과 언어 모델 재사용하기

In [8]:
import os
import tensorflow_hub as hub


os.environ['TFHUB_CACHE_DIR'] = 'my_tfhub_cache'
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit(train_set, validation_data=valid_set, epochs=10)

2023-10-30 14:08:26.220974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10


2023-10-30 14:08:34.643137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 